# Speculative Decoding

SGLang now provides an EAGLE-based speculative decoding option. The implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

**Note:** Currently, Speculative Decoding in SGLang does not support radix cache.

### Performance Highlights

- Official EAGLE code ([SafeAILab/EAGLE](https://github.com/SafeAILab/EAGLE)): ~200 tokens/s
- Standard SGLang Decoding: ~156 tokens/s
- EAGLE Decoding in SGLang: ~297 tokens/s
- EAGLE Decoding in SGLang (w/ `torch.compile`): ~316 tokens/s

All benchmarks below were run on a single H100.

## EAGLE Decoding

To enable EAGLE-based speculative decoding, specify the draft model (`--speculative-draft-model-path`) and the relevant EAGLE parameters:

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmzheng/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-03-03 03:35:09] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='127.0.0.1', port=32001, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=True, tp_size=1, stream_interval=1, stream_output=False, random_seed=670741452, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, show_time_cost=Fals

[2025-03-03 03:35:28 TP0] Init torch distributed begin.
[2025-03-03 03:35:28 TP0] Load weight begin. avail mem=60.63 GB


[2025-03-03 03:35:28 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-03 03:35:30 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.36s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.36s/it]

[2025-03-03 03:35:35 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=36.35 GB
[2025-03-03 03:35:35 TP0] KV Cache is allocated. K size: 0.16 GB, V size: 0.16 GB.
[2025-03-03 03:35:35 TP0] Memory pool end. avail mem=36.03 GB


[2025-03-03 03:35:35 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=4096
[2025-03-03 03:35:35] INFO:     Started server process [1616878]
[2025-03-03 03:35:35] INFO:     Waiting for application startup.
[2025-03-03 03:35:35] INFO:     Application startup complete.
[2025-03-03 03:35:35] INFO:     Uvicorn running on http://127.0.0.1:32001 (Press CTRL+C to quit)


[2025-03-03 03:35:36] INFO:     127.0.0.1:47200 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-03 03:35:36] INFO:     127.0.0.1:47202 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-03 03:35:36 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 03:35:40] INFO:     127.0.0.1:47212 - "POST /generate HTTP/1.1" 200 OK
[2025-03-03 03:35:40] The server is fired up and ready to roll!


In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-03 03:35:41 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 03:35:41] INFO:     127.0.0.1:46786 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [3]:
terminate_process(server_process)

### EAGLE Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--cuda-graph-max-bs`:


In [4]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmzheng/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --cuda-graph-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-03-03 03:35:55] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='127.0.0.1', port=36215, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=True, tp_size=1, stream_interval=1, stream_output=False, random_seed=860706113, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False

[2025-03-03 03:36:14 TP0] Init torch distributed begin.
[2025-03-03 03:36:14 TP0] Load weight begin. avail mem=78.81 GB


[2025-03-03 03:36:14 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-03 03:36:15 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.28s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.28s/it]

[2025-03-03 03:36:20 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=38.91 GB
[2025-03-03 03:36:20 TP0] KV Cache is allocated. K size: 0.16 GB, V size: 0.16 GB.
[2025-03-03 03:36:20 TP0] Memory pool end. avail mem=38.59 GB


[2025-03-03 03:36:20 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=4096
[2025-03-03 03:36:20] INFO:     Started server process [1619469]
[2025-03-03 03:36:20] INFO:     Waiting for application startup.
[2025-03-03 03:36:20] INFO:     Application startup complete.
[2025-03-03 03:36:20] INFO:     Uvicorn running on http://127.0.0.1:36215 (Press CTRL+C to quit)


[2025-03-03 03:36:20] INFO:     127.0.0.1:45510 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-03 03:36:21] INFO:     127.0.0.1:41310 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-03 03:36:21 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 03:36:24] INFO:     127.0.0.1:41318 - "POST /generate HTTP/1.1" 200 OK
[2025-03-03 03:36:24] The server is fired up and ready to roll!


In [5]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-03 03:36:25 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 03:36:26] INFO:     127.0.0.1:41330 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [6]:
terminate_process(server_process)

### EAGLE Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [7]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmzheng/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-03-03 03:36:40] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='127.0.0.1', port=35330, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=True, tp_size=1, stream_interval=1, stream_output=False, random_seed=218840376, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False,

[2025-03-03 03:36:45] Casting torch.bfloat16 to torch.float16.


[2025-03-03 03:36:59 TP0] Casting torch.bfloat16 to torch.float16.


[2025-03-03 03:37:00 TP0] Casting torch.bfloat16 to torch.float16.
[2025-03-03 03:37:00 TP0] Init torch distributed begin.
[2025-03-03 03:37:00 TP0] Load weight begin. avail mem=78.81 GB


[2025-03-03 03:37:00 TP0] The following error message 'operation scheduled before its operands' can be ignored.


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.14s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.14s/it]

[2025-03-03 03:37:18 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=43.23 GB
[2025-03-03 03:37:18 TP0] KV Cache is allocated. K size: 0.04 GB, V size: 0.04 GB.
[2025-03-03 03:37:18 TP0] Memory pool end. avail mem=43.15 GB


[2025-03-03 03:37:19 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=8192
[2025-03-03 03:37:19] INFO:     Started server process [1622246]
[2025-03-03 03:37:19] INFO:     Waiting for application startup.
[2025-03-03 03:37:19] INFO:     Application startup complete.
[2025-03-03 03:37:19] INFO:     Uvicorn running on http://127.0.0.1:35330 (Press CTRL+C to quit)


[2025-03-03 03:37:19] INFO:     127.0.0.1:43188 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-03 03:37:20] INFO:     127.0.0.1:43200 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-03 03:37:20 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 03:37:23] INFO:     127.0.0.1:43214 - "POST /generate HTTP/1.1" 200 OK
[2025-03-03 03:37:23] The server is fired up and ready to roll!


In [8]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-03-03 03:37:24 TP0] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-03-03 03:37:25] INFO:     127.0.0.1:54608 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [9]:
terminate_process(server_process)